In [ ]:
# ==============================
#  Градиентный бустинг на CatBoost
#  с подробными пояснениями каждой строки
# ==============================

import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
# 1. Генерируем синтетические данные (можно заменить на свои)
X = np.random.randn(5000, 20)                    # 5000 объектов, 20 числовых признаков
y = (X[:, 0] > 0.5).astype(int) * 8 + \
    X[:, 1]**2 + \
    np.sin(X[:, 2] * 5) + \
    np.random.normal(0, 0.5, 5000)                # нелинейная зависимость + шум

# Добавим несколько категориальных признаков (CatBoost умеет с ними работать нативно)
cat_features = np.random.choice(['A', 'B', 'C', 'D'], size=5000)
X = np.column_stack([X, cat_features])           # последний столбец — категориальный

In [ ]:
# 2. Разбиваем на train / test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# 3. Создаём Pool — специальный объект CatBoost, который умеет хранить категориальные признаки
train_pool = Pool(X_train, y_train, cat_features=[20])   # индекс категориального признака = 20
test_pool  = Pool(X_test,  y_test,  cat_features=[20])

In [ ]:
# 4. Создаём модель
model = CatBoostRegressor(
    iterations=1000,          # максимальное число деревьев (M)
    learning_rate=0.05,       # η — shrinkage, самый важный параметр регуляризации
    depth=6,                  # глубина каждого дерева (обычно 4–10)
    l2_leaf_reg=3,            # ℓ₂-регуляризация на веса листьев (lambda)
    border_count=128,         # число бинов для числовых признаков (чем больше — точнее, но медленнее)
    bagging_temperature=1,    # стохастичность подвыборок (0 = без стохастики, 1 = как в Random Forest)
    random_strength=1,        # случайность при выборе сплита
    loss_function='RMSE',     # функция потерь (можно Logloss, Poisson, MAE и др.)
    eval_metric='RMSE',       # метрика, которую будем отслеживать
    verbose=50,               # выводим лог каждые 50 итераций
    early_stopping_rounds=100,# если 100 итераций нет улучшения на валидации — стоп
    use_best_model=True,      # после остановки вернём модель с лучшим результатом
    random_seed=42
)

In [ ]:
# 5. Обучаем
#    eval_set — валидационная выборка, по ней считается early stopping
model.fit(
    train_pool,
    eval_set=test_pool,
    plot=False                # можно True — откроется интерактивный график в Jupyter
)

In [ ]:
# 6. Предсказываем и считаем метрику
preds = model.predict(X_test)
print(f"Test RMSE = {mean_squared_error(y_test, preds, squared=False):.4f}")

In [ ]:
# 7. Важность признаков (CatBoost умеет считать SHAP и встроенную важность)
importances = model.get_feature_importance(type='FeatureImportance')
feature_names = [f'feature_{i}' for i in range(20)] + ['category']
for name, imp in zip(feature_names, importances):
    if imp > 1.0:
        print(f"{name}: {imp:.2f}")

In [ ]:
# 8. Сохраняем модель (если нужно)
model.save_model("catboost_model.cbm")


In [ ]:
# 9. Как загрузить обратно
loaded_model = CatBoostRegressor()
loaded_model.load_model("catboost_model.cbm")